<center><font size=6>Diagnose latent Cytomegalovirus using Deep learning</font></center>
.

<center>Zicheng Hu, Ph.D.</center>
<center>The Unversity of California, San Francisco</center>

In this tutorial, we will achieve the following goals:
1. Learn how to use **keras**, a popular deep learning framework.
1. Construct a deep learning model tailered to cytometry data. 
1. Apply the deep learning model to diagnose latent cytomegalovirus (CMV) infection. 
1. Examine and interpret the internal layers of the deep learning model. 

# Get Data

The dataset for this tutorial has to be downloaded. We check to see if the dataset has already been downloaded, and if not we download it.

In [ ]:
tutorial_files = ! ls
if "allData.obj" not in tutorial_files:
    print("Downloading Data:")
    ! wget https://storage.googleapis.com/public-files-900/Zicheng_Tutorial/allData.obj

# Main Tutorial

### Import libraries

Before we start, we first import libraries that we will use in this tutorial, including:
* **keras**, a library the handles the user-end of deep learning. 
* **tensorflow**, a library that handles the back-end of deep learning.
* **pickle**, a library for loading data.
* **pandas**, a library for manipulating data. 
* **random**, a library for generating random numbers. 
* **numpy**, a library for matrix computation. 
* **matplotlib**, a library for data visualization
* **sklearn**, a library that provides infranstructure of machine learning.

In [ ]:
##### import functions #####
from __future__ import print_function
from tensorflow.python.keras.layers import Dense, Flatten, BatchNormalization, Activation
from tensorflow.python.keras.layers import Conv2D, AveragePooling2D, Input
from tensorflow.python.keras.models import load_model, Model
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras import backend as K
import pickle
import pandas as pd
import numpy as np
from numpy.random import seed; seed(111)
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import tensorflow as tf

In [ ]:
tf.random.set_seed(111)

### Load data

First, we will load the data, which are stored in the "allData.obj" file. For the convinience of this tutorial, the data are downloaded from the ImmPort database and are already preprocessed. The data includes three parts, meta-data, CyTOF data and marker names. 
* The meta-data contains the demographic information of each subject, the study acession number for each sample, and the ground truth of CMV infection. It is stored as a pandas data frame. 
* The CyTOF data contains the single-cell profile of 27 markers. It is stored in a three-dimentional numpy array. The dimension of the numpy array is : 472 samples x 10000 cells x 27 markers. 
* The marker names contains the name of the 27 makers.

In [ ]:
##### load data #####
allData = pickle.load( open( "allData.obj", "rb" ) )
metaData = allData["metaData"]
cytoData = allData["cytoData"]
markerNames = allData["markerNames"]
print("Dimension of cytoData: ",cytoData.shape)
print("Names of the 27 makers: \n",markerNames.values)

### split data into training, validation and testing sets

Now, lets split the data into training, validation and testing sets. The training data is used to train the deep learning model. The validation dataset is used to select the best parameters for the model and to avoid overfitting. The test dataset is used to evaluate the performance of the deep learning model.

We will use samples from the study SDY515 as validation set, samples from the study SDY519 as testing set, and the rest samples as training samples. 

In [ ]:
##### split train, validation and test######
y = metaData.CMV_Ab.values
x = cytoData

train_id = (metaData.study_accession.isin(["SDY515","SDY519"])==False)
valid_id = metaData.study_accession=="SDY515"
test_id = metaData.study_accession =="SDY519"

x_train = x[train_id]; y_train = y[train_id]
x_valid = x[valid_id]; y_valid = y[valid_id]
x_test = x[test_id]; y_test = y[test_id]

### Define deep learing model


We will use a customized convolution neural network (CNN) to analyze the CyTOF data. For each sample, the CyTOF data is a matrix with rows as cells and columns as markers. It is important to notice that the order of cells is arbitry in CyTOF data. For example, both matrix 1 and matrix 2 profiles the sample sample in Figure 1. 

Based on the characterisitics of the CyTOF data, we can design a CNN model that use the raw cytof data as input, without the need of cell gating or cell clustering. The model contains 6 layers: input layer, first and second convolution layer, pooling layer, dense layer, and output layer. 


**Define Layers**

In [ ]:

# input layer
model_input = Input(shape=x_train[0].shape)

# first convolution layer
model_output = Conv2D(3, kernel_size=(1, x_train.shape[2]),
                 activation=None)(model_input)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# sceond convolution layer
model_output = Conv2D(3, (1, 1), activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# pooling layer
model_output = AveragePooling2D(pool_size=(x_train.shape[1], 1))(model_output)
model_output = Flatten()(model_output)

# Dense layer
model_output = Dense(3, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# output layer
model_output = Dense(1, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("sigmoid")(model_output)

**define model**

In [ ]:

model = Model(inputs=[model_input],
              outputs=model_output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='saved_weights.hdf5', 
                               monitor='val_loss', verbose=0, 
                               save_best_only=True, save_weights_only = True)

earlyStop = EarlyStopping(monitor='loss', min_delta=0.00000001, 
                          patience=100, verbose=0, mode='auto', 
                          baseline=None, restore_best_weights=True)


**Fit model**

In [ ]:

model.fit([x_train], y_train,
          batch_size=60,
          epochs=10,
          verbose=1,
          callbacks= [checkpointer, earlyStop], #[checkpointer, earlyStop], 
          validation_data=([x_valid], y_valid))

## Analysis of Results

In [ ]:
# plot train and validation loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
##### test the final model #####
best_model = load_model('Final_weights.hdf5')
y_scores = best_model.predict([x_test])
test_auc = roc_auc_score(y_true=y_test, y_score=y_scores)
print('The Area Under the Curve (AUC) = {0:.2f}'.format(test_auc))

In [ ]:
##### get activation value #####
get_6th_layer_output = K.function([best_model.layers[0].input],
                                  [best_model.layers[6].output])
sixth_layer_output = get_6th_layer_output([x_test])[0]

plot_df = x_test.reshape((x_test.shape[0]*x_test.shape[1],
                          x_test.shape[2]))
plot_df = pd.DataFrame(plot_df,columns=markerNames)

activation = sixth_layer_output[:,:,:,0]
plot_df["activation"] = (activation.reshape((activation.shape[0]*activation.shape[1],
                          activation.shape[2])))

In [ ]:
##### 
plot_df2 = plot_df.sample(10000)

sns.scatterplot(x=plot_df2["CD8"], y=plot_df2["CD3"], 
                hue=(plot_df2["activation"]>0.5),s = 10)
